<a href="https://colab.research.google.com/github/FathirAMM/Celetial_Minds_Predicta_1.0/blob/main/Challange_2_Weather_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi  # this should display information about available GPUs

Sun Jun 23 12:16:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#%load_ext cudf.pandas #nvidia cudf

In [2]:
import pandas as pd

In [ ]:
#import pandas as pd

# Load the historical weather data
file_path = '/content/daily_data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset and summary information to understand its structure
data.head()

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise,sunset
0,D0001,C001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,06:04 AM,07:19 PM
1,D0002,C001,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,06:05 AM,07:18 PM
2,D0003,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,06:05 AM,07:18 PM
3,D0004,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,06:06 AM,07:16 PM
4,D0005,C001,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,10.0,1.0,9.0,1,06:07 AM,07:15 PM


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2893 entries, 0 to 2892
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   day_id                    2893 non-null   object 
 1   city_id                   2893 non-null   object 
 2   temperature_celsius       2893 non-null   float64
 3   condition_text            479 non-null    object 
 4   wind_kph                  2893 non-null   float64
 5   wind_degree               2893 non-null   int64  
 6   pressure_mb               2893 non-null   float64
 7   precip_mm                 2893 non-null   float64
 8   humidity                  2893 non-null   int64  
 9   cloud                     2893 non-null   int64  
 10  feels_like_celsius        2893 non-null   float64
 11  visibility_km             2893 non-null   float64
 12  uv_index                  2893 non-null   float64
 13  gust_kph                  2893 non-null   float64
 14  air_qual

In [ ]:
# prompt: unique condition_text and there numbers

unique_conditions = data['condition_text'].unique()
condition_counts = data['condition_text'].value_counts()

print("Unique weather conditions:")
for condition in unique_conditions:
  print(f"- {condition}")

print("\nNumber of occurrences for each condition:")
for condition, count in condition_counts.items():
  print(f"- {condition}: {count}")


Unique weather conditions:
- nan
- Light Rain with Thunder
- Clear and Sunny
- Partly Cloudy
- Light Precipitation
- Cloudy and Overcast
- Mist or Fog
- Rain Showers
- Moderate to Heavy Rain
- Thunderstorms

Number of occurrences for each condition:
- Clear and Sunny: 122
- Partly Cloudy: 122
- Light Precipitation: 68
- Cloudy and Overcast: 53
- Mist or Fog: 50
- Rain Showers: 21
- Light Rain with Thunder: 16
- Thunderstorms: 15
- Moderate to Heavy Rain: 12


In [ ]:
data.isnull().sum()

day_id                         0
city_id                        0
temperature_celsius            0
condition_text              2414
wind_kph                       0
wind_degree                    0
pressure_mb                    0
precip_mm                      0
humidity                       0
cloud                          0
feels_like_celsius             0
visibility_km                  0
uv_index                       0
gust_kph                       0
air_quality_us-epa-index       0
sunrise                        0
sunset                         0
dtype: int64

In [ ]:
data.describe()

,temperature_celsius,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index
count,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000
mean,22.024577,10.304805,158.994469,1013.745938,0.165299,74.773591,32.073280,23.725752,9.745524,2.216730,16.930522,1.599032
std,6.615540,7.264472,103.021634,6.172045,0.937061,19.335724,32.584184,8.877424,2.489918,2.238163,10.541449,1.062513
min,-2.000000,3.600000,1.000000,982.000000,0.000000,4.000000,0.000000,-4.300000,0.000000,1.000000,0.700000,1.000000
25%,17.000000,5.000000,70.000000,1010.000000,0.000000,66.000000,0.000000,17.000000,10.000000,1.000000,9.000000,1.000000
50%,22.000000,8.300000,150.000000,1014.000000,0.000000,78.000000,25.000000,24.500000,10.000000,1.000000,15.100000,1.000000
75%,27.000000,13.000000,236.000000,1018.000000,0.000000,89.000000,68.000000,30.000000,10.000000,1.000000,22.300000,2.000000
max,45.000000,74.200000,360.000000,1036.000000,28.700000,100.000000,100.000000,73.600000,32.000000,10.000000,75.600000,6.000000


In [ ]:
# Import libraries
import pandas as pd  # Pandas for data manipulation
from sklearn.preprocessing import LabelEncoder  # Encode categorical labels
from sklearn.ensemble import GradientBoostingClassifier  # Classification model
from sklearn.model_selection import RandomizedSearchCV  # Hyperparameter tuning
from datetime import datetime  # Work with dates and times

# Load the dataset
file_path = 'daily_data.csv'
data = pd.read_csv(file_path)  # Read data from 'daily_data.csv'


In [ ]:
# Convert 'sunrise' and 'sunset' times to seconds from midnight
'''
convert_to_seconds function converts a time string (e.g., '6:30 AM') to seconds since midnight.
The apply function is used to convert all entries in the 'sunrise' and 'sunset' columns.
'''
# Function to convert time to seconds from midnight
def convert_to_seconds(time_str):
    t = datetime.strptime(time_str, '%I:%M %p').time()  # Parse the time string
    return t.hour * 3600 + t.minute * 60 + t.second  # Convert to seconds

# Apply the conversion to the 'sunrise' and 'sunset' columns
data['sunrise'] = data['sunrise'].apply(convert_to_seconds)
data['sunset'] = data['sunset'].apply(convert_to_seconds)


In [ ]:
#  Create a new feature: duration of daylight
'''
A new column daylight_duration is added, representing the difference between sunset and sunrise times in seconds.
'''
# Calculate daylight duration as the difference between sunset and sunrise
data['daylight_duration'] = data['sunset'] - data['sunrise']

In [ ]:
# Handle missing values in 'condition_text' and encode the categorical labels

In [ ]:
# Fill missing values in 'condition_text' with 'Unknown'
'''
Missing values in 'condition_text' are filled with 'Unknown'.
The LabelEncoder encodes the 'condition_text' column into numerical labels for model training.
'''
data['condition_text'].fillna('Unknown', inplace=True)

# Encode 'condition_text' as categorical values
label_encoder = LabelEncoder()
data['condition_text'] = label_encoder.fit_transform(data['condition_text'])


In [ ]:
# Prepare feature set and target variable
'''
Unnecessary columns 'day_id', 'city_id', and 'condition_text' are dropped from the features set X.
y is defined as the target variable containing the encoded condition labels.
'''
# Drop unnecessary columns and separate features and target
X = data.drop(columns=['day_id', 'city_id', 'condition_text'])
y = data['condition_text']


In [ ]:
# Split the data into training and prediction sets
'''
train_data excludes rows where 'condition_text' was 'Unknown'.
X_train and y_train are the features and target variables for the training set.
Shapes and columns of X_train and y_train are printed to verify the data.
'''
# Split the data into training and prediction sets
# Exclude 'Unknown' labels from training data
train_data = data[data['condition_text'] != label_encoder.transform(['Unknown'])[0]]
X_train = train_data.drop(columns=['day_id', 'city_id', 'condition_text'])
y_train = train_data['condition_text']

# Print shapes and columns to verify data integrity
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_train columns: {X_train.columns}')


In [ ]:
# Prepare the prediction data set.
'''
predict_data includes only rows where 'condition_text' is 'Unknown'.
X_predict is prepared for prediction by dropping unnecessary columns.
'''
# Define the dataset for prediction (where 'condition_text' is 'Unknown')
predict_data = data[data['condition_text'] == label_encoder.transform(['Unknown'])[0]]
X_predict = predict_data.drop(columns=['day_id', 'city_id', 'condition_text'])


In [ ]:
# Define and train the model with hyperparameter tuning
'''
GradientBoostingClassifier is initialized with a fixed random seed for reproducibility.
param_grid defines the hyperparameters to search over.
RandomizedSearchCV performs a randomized search over the hyperparameters.
The model is trained using fit on the training data.
The best model from the search is obtained using best_estimator_.
'''
# Initialize the Gradient Boosting Classifier
model = GradientBoostingClassifier(random_state=42)

# Define the hyperparameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0]
}

# Initialize RandomizedSearchCV with the model and parameter grid
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                                   n_iter=50, scoring='accuracy', cv=3,
                                   verbose=1, random_state=42, n_jobs=-1)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train, y_train)

# Get the best model from RandomizedSearchCV
best_model = random_search.best_estimator_


In [ ]:
# Predict the missing 'condition_text' values
'''
Predictions for 'Unknown' condition texts are made using the best model.
The predicted labels are decoded back to their original categorical values.
'''
# Predict the 'Unknown' condition_text values using the best model
y_predict = best_model.predict(X_predict)

# Decode the predicted labels back to original condition_text values
predicted_conditions = label_encoder.inverse_transform(y_predict)


In [ ]:
# Fill predictions into the original data and save the results
'''
The original data is updated with the predicted 'condition_text' values.
A submission DataFrame full_submission is created containing 'day_id' and the decoded 'condition_text'.
The results are saved to a CSV file 'full_submission_improved.csv'.
The first few rows of the submission DataFrame are printed to verify the output.
'''
# Update the original data with the predicted values for 'Unknown' conditions
data.loc[data['condition_text'] == label_encoder.transform(['Unknown'])[0], 'condition_text'] = y_predict

# Prepare the submission DataFrame with 'day_id' and 'condition_text'
full_submission = data[['day_id', 'condition_text']]
full_submission['condition_text'] = label_encoder.inverse_transform(full_submission['condition_text'])

# Save the submission DataFrame to a CSV file
full_submission_file_path = 'full_submission_improved.csv'
full_submission.to_csv(full_submission_file_path, index=False)

# Display the first few rows of the final submission file
print(full_submission.head())


All in one

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime

# Load the dataset
file_path = 'daily_data.csv'
data = pd.read_csv(file_path)

# Convert 'sunrise' and 'sunset' to datetime and then to seconds from midnight
def convert_to_seconds(time_str):
    t = datetime.strptime(time_str, '%I:%M %p').time()
    return t.hour * 3600 + t.minute * 60 + t.second

data['sunrise'] = data['sunrise'].apply(convert_to_seconds)
data['sunset'] = data['sunset'].apply(convert_to_seconds)

# Create a new feature: duration of daylight
data['daylight_duration'] = data['sunset'] - data['sunrise']

# Fill NA in condition_text column with 'Unknown' for the purpose of label encoding
data['condition_text'].fillna('Unknown', inplace=True)

# Encode 'condition_text' as categorical
label_encoder = LabelEncoder()
data['condition_text'] = label_encoder.fit_transform(data['condition_text'])

# Prepare feature set and target variable
X = data.drop(columns=['day_id', 'city_id', 'condition_text'])
y = data['condition_text']

# Split the data into training and testing sets, keeping 'Unknown' out of the training set
train_data = data[data['condition_text'] != label_encoder.transform(['Unknown'])[0]]
X_train = train_data.drop(columns=['day_id', 'city_id', 'condition_text'])
y_train = train_data['condition_text']

# Verify the data integrity
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_train columns: {X_train.columns}')

# Define a separate set for prediction
predict_data = data[data['condition_text'] == label_encoder.transform(['Unknown'])[0]]
X_predict = predict_data.drop(columns=['day_id', 'city_id', 'condition_text'])

# Define the model
model = GradientBoostingClassifier(random_state=42)

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0]
}

# Define the random search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                                   n_iter=50, scoring='accuracy', cv=3,
                                   verbose=1, random_state=42, n_jobs=-1)

# Fit the random search model
random_search.fit(X_train, y_train)

# Get the best model from the random search
best_model = random_search.best_estimator_

# Predict the missing 'condition_text' values
y_predict = best_model.predict(X_predict)

# Decode the predicted labels back to original condition_text
predicted_conditions = label_encoder.inverse_transform(y_predict)

# Fill the predictions into the original data
data.loc[data['condition_text'] == label_encoder.transform(['Unknown'])[0], 'condition_text'] = y_predict

# Create the submission DataFrame
full_submission = data[['day_id', 'condition_text']]
full_submission['condition_text'] = label_encoder.inverse_transform(full_submission['condition_text'])

# Save the full submission DataFrame to a CSV file
full_submission_file_path = 'full_submission_improved.csv'
full_submission.to_csv(full_submission_file_path, index=False)

# Display the first few rows of the final submission file
print(full_submission.head())


X_train shape: (479, 15)
y_train shape: (479,)
X_train columns: Index(['temperature_celsius', 'wind_kph', 'wind_degree', 'pressure_mb',
       'precip_mm', 'humidity', 'cloud', 'feels_like_celsius', 'visibility_km',
       'uv_index', 'gust_kph', 'air_quality_us-epa-index', 'sunrise', 'sunset',
       'daylight_duration'],
      dtype='object')
Fitting 3 folds for each of 50 candidates, totalling 150 fits
  day_id           condition_text
0  D0001            Partly Cloudy
1  D0002            Partly Cloudy
2  D0003  Light Rain with Thunder
3  D0004          Clear and Sunny
4  D0005          Clear and Sunny


<ipython-input-4-7473b4172886>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_submission['condition_text'] = label_encoder.inverse_transform(full_submission['condition_text'])
